In [ ]:
import numpy as np
import keras
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D
from keras.utils import plot_model
from keras.datasets import mnist

In [ ]:
# Załaduj dane
(x_train,y_train), (x_test, y_test) = mnist.load_data()
# podgląd
print("etykieta ",y_train[0])
plt.imshow(x_train[0], cmap='gray')

In [ ]:
# Przetwórz dane
# Dane typu float32, modele często lepiej radzą sobie z danymi liczbowymi w formie zmiennoprzecinkowej.
# Normalizujemy dane do przedziału [0, 1]
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

In [ ]:
# Konwertuj etykiety na kategorie
# kodowanie kategoryczne (one-hot encoding) na zbiorze etykiet treningowych
# zamienia etykiety klas na postać binarną w formie wektorów zer i jedynek
# 3 -> [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
# Zdefiniuj model za pomocą Functional API
# Conv2D(liczba filtrów,...)
input_shape = (28, 28, 1)
inputs = Input(shape=input_shape)
x = Conv2D(4, kernel_size=(3, 3), activation='relu')(inputs)#64
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)#128
outputs = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=inputs, outputs=outputs)

In [ ]:
# Skompiluj model
# 'categorical_crossentropy' średnia z logarytmu przewidywanych prawdopodobieństw dla prawdziwej klasy
# Optymalizator Adam (Adaptive Moment Estimation) wykorzystuje adaptacyjne momenty gradientu do efektywnego dostosowywania wag modelu podczas treningu
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# Trenuj model
# batch_size liczba próbek treningowych użytych do jednej aktualizacji wag modelu podczas jednej iteracji treningowej
model.fit(x_train, y_train, batch_size=128, epochs=5, verbose='auto', validation_data=(x_test, y_test))

In [ ]:
# Ocena modelu
loss, accuracy = model.evaluate(x_test, y_test, verbose='auto')
print('Test loss:', loss)
print('Test accuracy:', accuracy)

In [ ]:
# Wizualizacja modelu
model.summary()
plot_model(model, show_shapes=True, show_layer_names=True)# to_file='model_plot.png',

In [ ]:
# Wyświetlanie przykładów źle sklasyfikowanych
predictions = model.predict(x_test)
incorrect_indices = np.nonzero(np.argmax(predictions, axis=1) != np.argmax(y_test, axis=1))[0]

for i in range(5):
    idx = incorrect_indices[i]
    print("Przykład źle sklasyfikowany nr", i+1)
    plt.imshow(x_test[idx], cmap='gray')
    plt.xlabel(f"True label: {np.argmax(y_test[idx])}, Predicted label: {np.argmax(predictions[idx])}")
    plt.show()